### Setup and import graph

In [2]:
######## setup variables ##########
gs_prefix="gs://anand-31-simple-files/customer-claims/customer-360"
#cluster_name="katana-internal31"
partitions = 2
prefix = gs_prefix
platform_prefix= prefix

print("--")

--


In [4]:
import os                                                                        
import time                                                                      
import json
import pandas as pd
from timeit import default_timer as timer
from katana.remote import aio, analytics, import_data, trace

from katana import remote
from katana.remote import import_data

from katana.remote.analytics import connected_components, louvain_clustering

# os.environ["KATANA_SERVER_ADDRESS"] = "host.docker.internal:8080"
os.environ["KATANA_SERVER_ADDRESS"] = "localhost:8080"

pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)

print("--")

--


In [5]:
rc=remote.Client()
rc.server_version

'0.3.0.dev+7539.7539.746661.746661'

In [6]:

graph = rc.create_graph(
    num_partitions=partitions,
    name="customer-claims"
)

print("--")

In [7]:
graph.graph_id

'CSDaoksyH5iGeqdiA4R8VLonh5hgnJxjEErTJizyNhiX'

In [9]:
test_node_file = platform_prefix + "/42_NodeHeader.txt"
test_edge_file = platform_prefix + "/43_EdgeHeader.txt"
input_path=platform_prefix 

print("--")

--


In [10]:
start = timer()
print("Importing graph from CSV files...")
import_data.csv(
    graph,
    input_node_path=test_node_file,
    input_edge_path=test_edge_file,
    input_dir=input_path,
#    have_headers=True,
    data_delimiter="|",
    schema_delimiter="|"
    #partition_policy="random-oec"
)
end = timer()
print(f"  import: {end - start:.1f} seconds") 

Importing graph from CSV files...


          0/? [?op/s]

  import: 51.7 seconds


In [11]:
%%time
graph.num_nodes()

          0/? [?op/s]

CPU times: user 110 ms, sys: 13.3 ms, total: 123 ms
Wall time: 6.52 s


2901419

In [12]:
graph.num_edges()

          0/? [?op/s]

4521599

## Investigate the neighbourhood of an Claimant with high payouts

In [13]:
%%time
#High number of payouts
query1="""
        MATCH (p:Claimant)<-[]-(py:Payment)
        RETURN count(py) as count, p.id order by count desc
        """
result = graph.query(query1, contextualize=True)
result.view()

          0/? [?op/s]

CPU times: user 1.78 s, sys: 131 ms, total: 1.91 s
Wall time: 7.57 s


<b> Let's have a look at the claimant neighborhood </b>

In [14]:
%%time
#Payments
query1="""
        MATCH (p:Claimant)<-[]-(py:Payment)
        where p.id = 'C_8900'
        RETURN p, py
        """
result = graph.query(query1, contextualize=True)
result.view()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 842 ms, sys: 75.5 ms, total: 918 ms
Wall time: 26.6 s


In [16]:
%%time
#check Folks involved in common claim
query1="""
        MATCH (p:Claimant)-[:MADE]->(c:Claim)<-[:MADE]-(p2)
        where p.id = 'C_8900'
        RETURN p, c, p2
        """
result = graph.query(query1, contextualize=True)
result.view()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 1.03 s, sys: 66.6 ms, total: 1.09 s
Wall time: 26.2 s


### Enriching the Topology: Duplicate Detection and Linking
We will pick claimants connected to C_8900 via common claims and look for duplicates

In [ ]:
%%time
query1="""
        MATCH (p:Claimant)-[]-(c:Claim)<-[]-(p2:Claimant)
        where p.id = 'C_8900'
        RETURN distinct(p2) order by p2.firstname
        """
result = graph.query(query1)
result.view()

In [ ]:
candidates = []
for e in result.iterrows():
    candidate = dict()
    candidate['firstname'] = e[1]['p2.firstname']
    candidate['lastname'] = e[1]['p2.lastname']
    candidate['id'] = e[1]['p2.id']
    candidate['address'] = e[1]['p2.address']
    candidates.append(candidate)

In [ ]:
len(candidates)

In [ ]:
import difflib
par1 = 'Daniel Williams'
par2 = 'Daniel William'
score = difflib.SequenceMatcher(None, par1, par2).ratio()
score

In [ ]:
def write_duplicates(graph, candidates):
    import csv
    import difflib
    print("Finding duplicates")
    import socket
    print(socket.gethostname())
    
    pos = int(socket.gethostname().replace('katana-compute-',''))
    lastIndex = len(candidates) - 1
    print('Last position of candidates to compare', lastIndex)
    duplicates=[]
    while pos < lastIndex:
        nameOfSource = candidates[pos]['firstname'] + candidates[pos]['lastname']
        addressOfSource = candidates[pos]['address']
        idOfSource = candidates[pos]['id']
        looper = 0
        while looper < lastIndex:
            idOfTarget = candidates[looper]['id']
            if idOfSource == idOfTarget:
                looper = looper + 1
                continue
            nameOfTarget = candidates[looper]['firstname'] + candidates[looper]['lastname']
            addressOfTarget = candidates[looper]['address']
            score = difflib.SequenceMatcher(None, nameOfSource, nameOfTarget).ratio()
            score_address = difflib.SequenceMatcher(None, addressOfSource, addressOfTarget).ratio()
            if(score > 0.8) and (score_address > 0.8):
                print('duplicates', idOfSource, idOfTarget)
                print('addressand score', addressOfSource, addressOfTarget, score_address)
                duplicate = dict()
                duplicate['idOfSource'] = idOfSource
                duplicate['idOfTarget'] = idOfTarget
                duplicates.append(duplicate)
            looper = looper + 1
        pos = pos + 5
    print('processing complete for node', socket.gethostname())
    print('last position compared', pos)
    
    outFileName = '/tmp/dups-1.csv'
    fields = ['idOfSource', 'idOfTarget']
    with open(outFileName, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fields, delimiter='|')
        writer.writeheader()
        if (len(duplicates) > 0): 
            writer.writerows(duplicates)
    print('finished writing duplicates')
    
#graph.run(lambda g: dist_count(g, 'argument1', 'argument2', 'argument3'))
#candidates = ['argument1', 'argument2', 'argument3','argument4', 'argument5']
graph.run(lambda g: write_duplicates(g, candidates))


In [ ]:
duplicates = []
duplicates.append(['C_312900','C_84200'])
duplicates.append(['C_236100','C_16800'])
duplicates.append(['C_106500','C_216400'])
len(duplicates)

In [8]:
query1="""
        MATCH (p1:Claimant),(p2:Claimant),(p3:Claimant),(p4:Claimant),(p5:Claimant),(p6:Claimant)
        where p1.id = '_d1_'
          and p2.id = '_d2_'
          and p3.id = '_d3_'
          and p4.id = '_d4_'
          and p5.id = '_d5_'
          and p6.id = '_d6_'
        with p1, p2, p3, p4, p5, p6
        create (p1)-[:isDuplicatedBy]->(p2)-[:isDuplicatedBy]->(p1),
               (p3)-[:isDuplicatedBy]->(p4)-[:isDuplicatedBy]->(p3),
               (p5)-[:isDuplicatedBy]->(p6)-[:isDuplicatedBy]->(p5)
        RETURN p1, p2, p3, p4, p5, p6
        """
query1 = query1.replace('_d1_', 'C_312900')
query1 = query1.replace('_d2_', 'C_84200')
query1 = query1.replace('_d3_', 'C_236100')
query1 = query1.replace('_d4_', 'C_16800')
query1 = query1.replace('_d5_', 'C_106500')
query1 = query1.replace('_d6_', 'C_216400')

print(query1)
result = graph.query(query1)


        MATCH (p1:Claimant),(p2:Claimant),(p3:Claimant),(p4:Claimant),(p5:Claimant),(p6:Claimant)
        where p1.id = 'C_312900'
          and p2.id = 'C_84200'
          and p3.id = 'C_236100'
          and p4.id = 'C_16800'
          and p5.id = 'C_106500'
          and p6.id = 'C_216400'
        with p1, p2, p3, p4, p5, p6
        create (p1)-[:isDuplicatedBy]->(p2)-[:isDuplicatedBy]->(p1),
               (p3)-[:isDuplicatedBy]->(p4)-[:isDuplicatedBy]->(p3),
               (p5)-[:isDuplicatedBy]->(p6)-[:isDuplicatedBy]->(p5)
        RETURN p1, p2, p3, p4, p5, p6
        


          0/? [?op/s]

In [ ]:
%%time
#check duplicate objects
query1="""
        MATCH (p1:Claimant)-[:isDuplicatedBy]->(p2:Claimant)
        RETURN p1, p2
        """
result = graph.query(query1, contextualize=True)
result.view()

## Revisit the neighborhood and check for a fraudulent patterns

In [17]:
%%time
#check Folks involved in common claim
query1="""
        MATCH (p:Claimant)-[:MADE]->(c:Claim)<-[e:MADE]-(p2)
        where p.id = 'C_8900'
        with p, c, p2
        MATCH (p:Claimant)-[:MADE]->(c:Claim)<-[e1:MADE]-(p2)
        where e1.ROLE = 'witness'
        RETURN p, c, p2
        """
result = graph.query(query1, contextualize=True)
result.view()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 1.07 s, sys: 71.4 ms, total: 1.14 s
Wall time: 26 s


## Project the claimant-claimant relationship, and then revisit the neighborhood of C_8900

In [18]:
%%time
query1="""
        MATCH (p1:Claimant)-[]-(c:Claim)-[]-(p2:Claimant)
        with p2,p1, count(c) as common_claim_count 
        create (p1)-[:common_claim{count: common_claim_count}]->(p2)-[:common_claim{count: common_claim_count}]->(p1)
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 569 ms, sys: 6.8 ms, total: 576 ms
Wall time: 23.4 s


## Pagerank and Louvain Algorithms

In [19]:
from katana.remote.analytics import pagerank
pagerank(graph, 'pg2')

          0/? [?op/s]

In [20]:
#reload the graph as needed
graph = rc.create_graph(
    num_partitions=partitions,
    name="customer_claims"
)

from katana.remote import import_data
import_data.rdg(graph, 'gs://katana-internal48/graph/' + 'AfGiMk4ah19yGsC88zhnpvxbinFrTtgXQusdxBkCQKei')
graph.graph_id

          0/? [?op/s]

'CMG9MRMEVwdV2hqydQYX5ebZHJGuhRywycXF1PMump4f'

In [21]:
gp2 = graph.project(node_types = ['Claimant'], edge_types = ['common_claim', 'isDuplicatedBy'])
louvain_clustering(gp2,'lv3',  is_symmetric=True)

          0/? [?op/s]

In [22]:
#reload the graph as needed
graph = rc.create_graph(
    num_partitions=partitions,
    name="customer_claims"
)

from katana.remote import import_data
import_data.rdg(graph, 'gs://katana-internal48/graph/' + '5fHrJV1gGBxXZEgsJJ3QeQY4bHbn4BodyZcdwKFDvSYR')
graph.graph_id

          0/? [?op/s]

'3s9iSvVzKgJmSgYgR9pYryC855ZWDeq6zPqUWgDhLNtc'

In [23]:
%%time
#check top clusters
query1="""
        MATCH (p:Claimant)
        with avg(toFloat(p.pg2))as avg_pr, count(p) as count, p.lv3 as lv
        where count > 1
        return avg_pr, count, lv order by avg_pr desc limit 10
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 194 ms, sys: 4.93 ms, total: 199 ms
Wall time: 24 s


In [ ]:
#visualize the top 3 communities

In [24]:
%%time
#contrast average payouts
query1="""
        MATCH (p:Claimant)
        with avg(toFloat(p.pg2))as avg_pr, count(p) as count, p.lv3 as lv
        where count > 1
        with avg_pr, count, lv order by avg_pr desc limit 20 
        match (x:Claimant)<-[:WAS_PAID]-(py:Payment)
        where tointeger(x.lv3) = lv
        return avg_pr, x.lv3, avg(toFloat(py)) as average_payout order by avg_pr desc limit 10
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 333 ms, sys: 41.5 ms, total: 375 ms
Wall time: 23.7 s


In [25]:
%%time
#Expand 3 cluster 
query1="""
        MATCH (p)-[]-(q)
        where
            tointeger(p.lv3) = 125223
        return p, q
        """
result = graph.query(query1, contextualize = True)
result.view()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 1.55 s, sys: 82.7 ms, total: 1.63 s
Wall time: 21.8 s


In [26]:
%%time
#contrast avg payment - 2
query1="""
        MATCH (p:Claimant)
        where exists(p.pg2)
        with p order by p.pg2 desc
        MATCH (c:Claim)<-[:MADE]-(p)<-[:WAS_PAID]-(py:Payment)
        return avg(py.amount) as average_payments_amount, p.id order by average_payments_amount desc limit 10
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 189 ms, sys: 7.75 ms, total: 197 ms
Wall time: 2.73 s


In [27]:
%%time
#check top nodes
query1="""
        MATCH (p:Claimant)
        where exists(p.pg2)
        RETURN p order by p.pg2 desc  limit 5
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 359 ms, sys: 10.4 ms, total: 369 ms
Wall time: 3.01 s


In [28]:
%%time
#check top nodes
query1="""
        MATCH (att:Attorney)
        where att.id = 'X_4853'
        RETURN att
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 373 ms, sys: 22.1 ms, total: 395 ms
Wall time: 8.13 s


In [29]:
%%time
#Get suspicious claims
query1="""
        MATCH (p1:Claimant)-[:isDuplicatedBy]-(p2:Claimant)-[MADE]-(cl:Claim)-[:REPRESENTS]->(att:Attorney)
        where att.id = 'X_4853'
        RETURN p1,p2,cl,att
        """
result = graph.query(query1, contextualize=True)
result.view()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 1.33 s, sys: 145 ms, total: 1.47 s
Wall time: 39.8 s


          0/? [?op/s]

'8SsgiQsdCKsjagFxciuAVScd2d6wRX9cBT5tdeXnGs3N'

## Page rank on projected graph

In [30]:
gp4 = graph.project(node_types = ['Claimant'], edge_types = ['common_claim', 'isDuplicatedBy'])
from katana.remote.analytics import pagerank
pagerank(gp4, 'pgp')

          0/? [?op/s]

In [31]:
%%time
#check top nodes
query1="""
        MATCH (p)
        where exists(p.pgp)
        RETURN p order by p.pgp desc  limit 5
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

OperationError: query BhHqvY5AJvTz4j4HDCT2co6bhCvH85oFZB8anFMy4KUM-GvSe8jJs6ZsTkZoi error: backtrace (QueryClient.cpp:395): backtrace (QueryClient.cpp:325): backtrace (GraphLifetime.cpp:250): loading graph for querying (GraphLifetime.cpp:283): backtrace (RDGLoader.h:43): backtrace (RDGLoader.cpp:446): backtrace (RDGLoader.cpp:466): no views found for requested version 2, rdg_uri file://katana-internal39/graph/3s9iSvVzKgJmSgYgR9pYryC855ZWDeq6zPqUWgDhLNtc (tsuba.cpp:205): not found

In [17]:
%%time
#check top nodes
query1="""
        MATCH (p)-[]-(q:Claim)
        where p.id = 'C_363900'
        RETURN p,q
        """
result = graph.query(query1, contextualize = True)
result.view()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 3.5 s, sys: 1.07 s, total: 4.56 s
Wall time: 25.5 s


In [31]:
%%time
#check transaction objects
query1="""
        MATCH (p:Claimant)
        where exists(p.pgp)
        with p order by p.pgp desc  limit 20
        Match(p:Claimant)-[]-(p1:Claimant)-[:isDuplicatedBy]-(p2:Claimant)
        with distinct(p)
        match (p:Claimant)-[:MADE]->(q:Claim)
        return p,q 
        """
result = graph.query(query1, contextualize=True)
result.view()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 1.87 s, sys: 519 ms, total: 2.39 s
Wall time: 26.1 s


In [32]:
%%time
#check transaction objects
query1="""
        MATCH (p:Claimant)
        where exists(p.pgp)
        with p order by p.pgp desc  limit 20
        Match(p:Claimant)-[]-(p1:Claimant)-[:isDuplicatedBy]-(p2:Claimant)
        with distinct(p)
        match (p:Claimant)-[:common_claim]->(q:Claimant)
        return p,q 
        """
result = graph.query(query1, contextualize=True)
result.view()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 2.77 s, sys: 961 ms, total: 3.73 s
Wall time: 24.4 s


### Louvain
run louvain + pagerank
produce a query with louvain cluster with average pagerank, sort by page rank desc, asc


In [13]:
gp2 = graph.project(node_types = ['Claimant'], edge_types = ['common_claim', 'isDuplicatedBy'])
louvain_clustering(gp2,'lv3',  is_symmetric=True)

          0/? [?op/s]

In [ ]:
%%time
#check transaction objects
query1="""
        MATCH (c:Claim)<-[:MADE]-(p:Claimant)<-[:WAS_PAID]-(py:Payment)
        return avg(py.amount) as average_payments_amount
        """
result = graph.query(query1)
result.view()


In [36]:
%%time
#check transaction objects
query1="""
        MATCH (p:Claimant)-[]-(q)
        where p.id = 'C_8900'
        RETURN p, q
        """
result = graph.query(query1, contextualize=True)
result.view()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 1.18 s, sys: 322 ms, total: 1.5 s
Wall time: 27 s


In [43]:
%%time
#check transaction objects
query1="""
        MATCH (p:Claimant)
        return sum(p.pg2 )as sum_pr, count(p) as count, p.lv3 as lv order by sum_pr desc limit 10
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 197 ms, sys: 72.8 ms, total: 270 ms
Wall time: 12.4 s


In [44]:
%%time
#check transaction objects
query1="""
        MATCH (p:Claimant)-[]-(b)
        where tointeger(p.lv3) = 2373
        RETURN p,b
        """
result = graph.query(query1, contextualize=True)
result.view()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 2.17 s, sys: 458 ms, total: 2.63 s
Wall time: 36.6 s


## Appendix

In [12]:
graph.graph_id

'EtUsiRFnqzyXf8iPvMzbK47fMok5JuGKvbvPJ41QyuEe'

In [16]:
%%time
#check transaction objects
query1="""
        MATCH (c:Claim)<-[:MADE]-(p:Claimant)<-[:WAS_PAID]-(py:Payment)
        return avg(py.amount) as average_payments_amount
        """
result = graph.query(query1)
result.view()


          0/? [?op/s]

CPU times: user 240 ms, sys: 63.3 ms, total: 304 ms
Wall time: 10.1 s


In [17]:
# add a query that produces page rank for average payment amount

          0/? [?op/s]

CPU times: user 377 ms, sys: 95.8 ms, total: 473 ms
Wall time: 10.3 s


## Claimants like C_231800 can be further investigated as they have nearly double the average payment going out

In [15]:
%%time
#check transaction objects
query1="""
        MATCH (p:Claimant)
        where p.id = 'C_231800'
        return p
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 464 ms, sys: 164 ms, total: 629 ms
Wall time: 10.4 s


In [16]:
%%time
#Claimants with high number of payments
query1="""
        MATCH (p:Claimant)
        where exists(p.pg2)
        with p order by p.pg2 desc
        MATCH (c:Claim)<-[:MADE]-(p)<-[:WAS_PAID]-(py:Payment)
        return count(py) as total_payments_made, p.id order by total_payments_made desc limit 10
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 369 ms, sys: 83.7 ms, total: 453 ms
Wall time: 5.29 s


### Other operations

In [39]:
from katana.remote.analytics import connected_components
connected_components(gp, 'cc2')

          0/? [?op/s]

In [40]:
%%time
#check transaction objects
query1="""
        MATCH (p:Claimant)
        where p.id = 'C_8900'
        RETURN p
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 755 ms, sys: 225 ms, total: 980 ms
Wall time: 14.8 s


In [41]:
%%time
#check transaction objects
query1="""
        MATCH (p1)-[]-(p2)
        where tointeger(p1.cc2) = 0
        return p1, p2
        """
result = graph.query(query1, contextualize = True)
result.view()

          0/? [?op/s]

CPU times: user 12.6 s, sys: 2.01 s, total: 14.6 s
Wall time: 53.1 s


In [45]:

%%time
#check transaction objects
query1="""
        CALL graph.schema() return *;
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 621 ms, sys: 227 ms, total: 848 ms
Wall time: 1min 28s


In [46]:
result.to_csv()

',neighbor,nodeType,properties\n0,,[Agent],"dob,email,firstname,id,lastname,pg2,ssn"\n1,([Location]::[IS_LOCATED_IN]),[Agent],\n2,,[Attorney],"dob,email,firstname,id,lastname,pg2,ssn"\n3,([Location]::[IS_LOCATED_IN]),[Attorney],\n4,,[Claim],"amount,attorney_id,capital_gains,capital_loss,claim_date,claimant_id,id,pg2,policy_id,witnesses"\n5,([Attorney]::[REPRESENTS]),[Claim],\n6,,[Claimant],"address,dob,email,firstname,id,lastname,lv3,pg2,ssn"\n7,([Claim]::[IS_LOCATED_IN]),[Claimant],\n8,([Claim]::[MADE]),[Claimant],ROLE\n9,([Claim]::[common_claim]),[Claimant],count\n10,([Claimant]::[MADE]),[Claimant],ROLE\n11,([Claimant]::[common_claim]),[Claimant],count\n12,([Claimant]::[isDuplicatedBy]),[Claimant],\n13,([Location]::[IS_LOCATED_IN]),[Claimant],\n14,([Location]::[common_claim]),[Claimant],count\n15,,[Location],"city,country,id,pg2,postalcode_plus4,state_abbr,street_address"\n16,,[Mbr],"amount,authorities_contacted,claim_id,id,incident_city,incident_date,incident_severity,incident_state